In [182]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/features/SelectedFeaturesVGGFinalOS.csv
/kaggle/input/features/FeatureStdVGGFinalTESTOS.csv
/kaggle/input/features/FeatureStdAlexFinalTESTOS.csv
/kaggle/input/features/SelectedFeaturesVGGFinalMergedMGMT.csv
/kaggle/input/features/FeatureAvgVGGFinal.csv
/kaggle/input/features/SelectedFeaturesVGGFinalMergedOS.csv
/kaggle/input/features/FeatureTargetAlexFinalTESTMGMT.csv
/kaggle/input/features/FeatureAvgVGGFinalTESTOS.csv
/kaggle/input/features/FeatureTargetVGGFinalMGMT.csv
/kaggle/input/features/SelectedFeaturesVGGFinal.csv
/kaggle/input/features/FeatureTargetVGGFinalTESTMGMT.csv
/kaggle/input/features/FeatureAvgAlexFinal.csv
/kaggle/input/features/FeatureAvgAlexFinalTESTMGMT.csv
/kaggle/input/features/SelectedFeaturesVGGFinalMerged.csv
/kaggle/input/features/AlexFinalMGMT.csv
/kaggle/input/features/SelectedFeaturesVGGFinalMergedTESTOS.csv
/kaggle/input/features/SelectedFeaturesAlexFinalMerged.csv
/kaggle/input/features/SelectedFeaturesVGGFinalTESTOS.csv
/kaggle/input/featu

In [183]:
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn import tree
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC
import numpy as np
import tensorflow as tf
tf.random.set_seed(42)

In [243]:
x_avg = pd.read_csv("/kaggle/input/features/FeatureAvgAlexFinalMGMT.csv")
x_std = pd.read_csv("/kaggle/input/features/FeatureStdAlexFinalMGMT.csv")
# y_train = pd.read_csv("/kaggle/input/features/FeatureTargetAlexFinalMGMT.csv")
y = pd.read_csv("/kaggle/input/features/FeatureTargetAlexFinalMGMT.csv")
g = pd.read_csv("/kaggle/input/features/TargetFinal.csv")

y = y.iloc[:, [1]]
# print(y['MGMT Status'].dtype)
y_train_all = pd.read_csv("/kaggle/input/features/TargetFinal.csv")
y_train = y_train_all.iloc[:, [1]]

test_df = pd.read_csv("/kaggle/input/features/test_df.csv")


x_avg_test = pd.read_csv("/kaggle/input/features/FeatureAvgAlexFinalTESTOS.csv")
x_std_test = pd.read_csv("/kaggle/input/features/FeatureStdAlexFinalTESTOS.csv")
y_train_test = pd.read_csv("/kaggle/input/features/FeatureTargetAlexFinalTESTOS.csv")


y_train_test = y_train_test.iloc[:, [1]]



In [244]:
g.head()

,ID,MGMT Status,OS
0,UPENN-GBM-00022_11,0,1
1,UPENN-GBM-00034_11,0,1
2,UPENN-GBM-00088_11,0,0
3,UPENN-GBM-00091_11,0,0
4,UPENN-GBM-00093_11,0,1


In [247]:
y_train_all.head()

,ID,MGMT Status,OS
0,UPENN-GBM-00022_11,0,1
1,UPENN-GBM-00034_11,0,1
2,UPENN-GBM-00088_11,0,0
3,UPENN-GBM-00091_11,0,0
4,UPENN-GBM-00093_11,0,1


In [242]:
x_avg.shape,y.shape,y_train.shape

((182, 1024), (182, 1), (182, 1))

In [186]:
x_avg.head()

,FLAIR_AlexNet_Avg_F0,FLAIR_AlexNet_Avg_F1,FLAIR_AlexNet_Avg_F2,FLAIR_AlexNet_Avg_F3,FLAIR_AlexNet_Avg_F4,FLAIR_AlexNet_Avg_F5,FLAIR_AlexNet_Avg_F6,FLAIR_AlexNet_Avg_F7,FLAIR_AlexNet_Avg_F8,FLAIR_AlexNet_Avg_F9,...,T2_AlexNet_Avg_F246,T2_AlexNet_Avg_F247,T2_AlexNet_Avg_F248,T2_AlexNet_Avg_F249,T2_AlexNet_Avg_F250,T2_AlexNet_Avg_F251,T2_AlexNet_Avg_F252,T2_AlexNet_Avg_F253,T2_AlexNet_Avg_F254,T2_AlexNet_Avg_F255
0,0.003814,0.045020,0.050345,0.031213,0.000000,0.174502,1.054971,0.024731,0.029978,0.189490,...,0.007143,0.016490,0.937989,0.150556,0.202410,0.493336,0.161363,0.972364,0.068355,0.069576
1,0.001027,0.222024,0.009933,0.007961,0.000093,0.007661,0.974528,0.001165,0.000350,0.333508,...,0.001419,0.006810,0.412239,0.145614,0.037569,0.156044,0.020090,0.670977,0.073361,0.113075
2,0.008434,0.076808,0.017377,0.021533,0.000657,0.093905,1.069724,0.009675,0.015556,0.093489,...,0.000304,0.005718,0.714271,0.142667,0.080807,0.175594,0.047335,0.839469,0.056587,0.080010
3,0.013356,0.035768,0.041434,0.007111,0.000719,0.048711,0.702616,0.004824,0.005732,0.268841,...,0.014847,0.004756,0.728194,0.114636,0.029974,0.316331,0.060670,0.332688,0.055494,0.015312
4,0.008370,0.163458,0.009650,0.019282,0.000222,0.067556,1.385033,0.006622,0.013892,0.136214,...,0.000812,0.005073,0.431138,0.041046,0.011424,0.252046,0.043557,0.859534,0.038391,0.037879


In [187]:
x_std.head()

,FLAIR_AlexNet_Std_F0,FLAIR_AlexNet_Std_F1,FLAIR_AlexNet_Std_F2,FLAIR_AlexNet_Std_F3,FLAIR_AlexNet_Std_F4,FLAIR_AlexNet_Std_F5,FLAIR_AlexNet_Std_F6,FLAIR_AlexNet_Std_F7,FLAIR_AlexNet_Std_F8,FLAIR_AlexNet_Std_F9,...,T2_AlexNet_Std_F246,T2_AlexNet_Std_F247,T2_AlexNet_Std_F248,T2_AlexNet_Std_F249,T2_AlexNet_Std_F250,T2_AlexNet_Std_F251,T2_AlexNet_Std_F252,T2_AlexNet_Std_F253,T2_AlexNet_Std_F254,T2_AlexNet_Std_F255
0,0.008332,0.056227,0.055364,0.031658,0.000000,0.095167,0.242279,0.031165,0.033846,0.127142,...,0.014506,0.020108,0.289769,0.087282,0.117931,0.268913,0.076862,0.220714,0.094536,0.068564
1,0.004032,0.134521,0.016267,0.009956,0.000549,0.009272,0.307101,0.002989,0.001493,0.191150,...,0.003654,0.013793,0.222223,0.095469,0.025701,0.039847,0.029178,0.166952,0.083168,0.083368
2,0.018597,0.116529,0.042113,0.024681,0.003309,0.069798,0.264216,0.030495,0.033947,0.079186,...,0.001313,0.011586,0.286517,0.052118,0.058948,0.094230,0.031335,0.180505,0.053220,0.095828
3,0.027287,0.058139,0.055558,0.011020,0.005083,0.045051,0.225321,0.012841,0.011289,0.133744,...,0.037402,0.007858,0.223235,0.081603,0.032951,0.107968,0.039944,0.145310,0.059187,0.020453
4,0.018329,0.121697,0.013141,0.039146,0.001016,0.050416,0.398619,0.014503,0.016751,0.081447,...,0.004549,0.007386,0.199021,0.029244,0.014775,0.184346,0.037175,0.136079,0.050477,0.039459


In [188]:
y_train.head()

,MGMT Status
0,0
1,0
2,0
3,0
4,0


In [189]:
from sklearn.preprocessing import StandardScaler
scaler_avg = StandardScaler()
scaler_std = StandardScaler()
scaler_avg_test = StandardScaler()
scaler_std_test = StandardScaler()

scaler_avg.fit(x_avg.values)
scaler_std.fit(x_std.values)

scaler_avg_test.fit(x_avg_test.values)
scaler_std_test.fit(x_std_test.values)

x_avg_test.shape

(200, 256)

In [190]:
len(scaler_avg.mean_)

1024

In [191]:
# scaler.transform(x_train)
x_avg = pd.DataFrame(scaler_avg.transform(x_avg.values), columns=x_avg.columns, index=x_avg.index)
x_std = pd.DataFrame(scaler_std.transform(x_std.values), columns=x_std.columns, index=x_std.index)

x_avg_test = pd.DataFrame(scaler_avg_test.transform(x_avg_test.values), columns=x_avg_test.columns, index=x_avg_test.index)
x_std_test = pd.DataFrame(scaler_std_test.transform(x_std_test.values), columns=x_std_test.columns, index=x_std_test.index)

x_avg.head()

,FLAIR_AlexNet_Avg_F0,FLAIR_AlexNet_Avg_F1,FLAIR_AlexNet_Avg_F2,FLAIR_AlexNet_Avg_F3,FLAIR_AlexNet_Avg_F4,FLAIR_AlexNet_Avg_F5,FLAIR_AlexNet_Avg_F6,FLAIR_AlexNet_Avg_F7,FLAIR_AlexNet_Avg_F8,FLAIR_AlexNet_Avg_F9,...,T2_AlexNet_Avg_F246,T2_AlexNet_Avg_F247,T2_AlexNet_Avg_F248,T2_AlexNet_Avg_F249,T2_AlexNet_Avg_F250,T2_AlexNet_Avg_F251,T2_AlexNet_Avg_F252,T2_AlexNet_Avg_F253,T2_AlexNet_Avg_F254,T2_AlexNet_Avg_F255
0,0.037443,-0.485992,-0.200800,1.229776,-0.490570,1.213024,-0.459453,0.518930,1.301485,-0.015855,...,-0.072879,0.483970,1.040061,0.190799,2.072027,1.382139,2.058795,1.758149,-0.523361,-0.369092
1,-0.525538,3.031056,-0.984724,-0.568099,-0.448785,-1.205605,-0.741583,-0.995751,-0.937361,0.922038,...,-0.577548,-0.416061,-1.014136,0.096650,-0.713844,-0.949756,-1.474351,0.163963,-0.427016,0.323514
2,0.970450,0.145625,-0.840313,0.481265,-0.194837,0.044633,-0.407710,-0.448778,0.211716,-0.641045,...,-0.675901,-0.517642,0.165956,0.040499,0.016894,-0.814591,-0.792951,1.055202,-0.749806,-0.202956
3,1.964570,-0.669838,-0.373655,-0.633878,-0.166905,-0.610526,-1.695235,-0.760595,-0.530652,0.500905,...,0.606487,-0.607086,0.220353,-0.493504,-0.842209,0.158402,-0.459452,-1.625420,-0.770847,-1.233110
4,0.957457,1.867359,-0.990214,0.307194,-0.390468,-0.337335,0.698145,-0.645004,0.085990,-0.362810,...,-0.631055,-0.577585,-0.940295,-1.895413,-1.155717,-0.286034,-0.887443,1.161333,-1.099970,-0.873788


In [192]:
x_avg.head()

,FLAIR_AlexNet_Avg_F0,FLAIR_AlexNet_Avg_F1,FLAIR_AlexNet_Avg_F2,FLAIR_AlexNet_Avg_F3,FLAIR_AlexNet_Avg_F4,FLAIR_AlexNet_Avg_F5,FLAIR_AlexNet_Avg_F6,FLAIR_AlexNet_Avg_F7,FLAIR_AlexNet_Avg_F8,FLAIR_AlexNet_Avg_F9,...,T2_AlexNet_Avg_F246,T2_AlexNet_Avg_F247,T2_AlexNet_Avg_F248,T2_AlexNet_Avg_F249,T2_AlexNet_Avg_F250,T2_AlexNet_Avg_F251,T2_AlexNet_Avg_F252,T2_AlexNet_Avg_F253,T2_AlexNet_Avg_F254,T2_AlexNet_Avg_F255
0,0.037443,-0.485992,-0.200800,1.229776,-0.490570,1.213024,-0.459453,0.518930,1.301485,-0.015855,...,-0.072879,0.483970,1.040061,0.190799,2.072027,1.382139,2.058795,1.758149,-0.523361,-0.369092
1,-0.525538,3.031056,-0.984724,-0.568099,-0.448785,-1.205605,-0.741583,-0.995751,-0.937361,0.922038,...,-0.577548,-0.416061,-1.014136,0.096650,-0.713844,-0.949756,-1.474351,0.163963,-0.427016,0.323514
2,0.970450,0.145625,-0.840313,0.481265,-0.194837,0.044633,-0.407710,-0.448778,0.211716,-0.641045,...,-0.675901,-0.517642,0.165956,0.040499,0.016894,-0.814591,-0.792951,1.055202,-0.749806,-0.202956
3,1.964570,-0.669838,-0.373655,-0.633878,-0.166905,-0.610526,-1.695235,-0.760595,-0.530652,0.500905,...,0.606487,-0.607086,0.220353,-0.493504,-0.842209,0.158402,-0.459452,-1.625420,-0.770847,-1.233110
4,0.957457,1.867359,-0.990214,0.307194,-0.390468,-0.337335,0.698145,-0.645004,0.085990,-0.362810,...,-0.631055,-0.577585,-0.940295,-1.895413,-1.155717,-0.286034,-0.887443,1.161333,-1.099970,-0.873788


In [193]:
x_std.head()

,FLAIR_AlexNet_Std_F0,FLAIR_AlexNet_Std_F1,FLAIR_AlexNet_Std_F2,FLAIR_AlexNet_Std_F3,FLAIR_AlexNet_Std_F4,FLAIR_AlexNet_Std_F5,FLAIR_AlexNet_Std_F6,FLAIR_AlexNet_Std_F7,FLAIR_AlexNet_Std_F8,FLAIR_AlexNet_Std_F9,...,T2_AlexNet_Std_F246,T2_AlexNet_Std_F247,T2_AlexNet_Std_F248,T2_AlexNet_Std_F249,T2_AlexNet_Std_F250,T2_AlexNet_Std_F251,T2_AlexNet_Std_F252,T2_AlexNet_Std_F253,T2_AlexNet_Std_F254,T2_AlexNet_Std_F255
0,-0.007439,-0.194127,-0.183687,1.195638,-0.669081,0.753232,-0.442680,0.403108,1.218581,0.222809,...,0.057928,0.378055,0.630467,0.749687,2.254330,1.877288,0.780112,0.447336,0.372754,-0.352574
1,-0.510895,2.026478,-1.121374,-0.671729,-0.533016,-1.232724,0.330073,-1.209368,-1.273089,1.173417,...,-0.689153,-0.186784,-0.241844,1.075643,-0.893295,-1.604976,-1.045401,-0.464956,0.070753,-0.034952
2,1.194345,1.516190,-0.501500,0.595289,0.151040,0.166687,-0.181162,0.364720,1.226359,-0.489400,...,-0.850276,-0.384158,0.588469,-0.650276,0.241355,-0.778252,-0.962808,-0.234963,-0.724817,0.232380
3,2.211840,-0.139910,-0.179023,-0.580170,0.590617,-0.405480,-0.644832,-0.645528,-0.518608,0.320861,...,1.634048,-0.717594,-0.228778,0.523602,-0.645847,-0.569399,-0.633229,-0.832195,-0.566291,-1.384750
4,1.162968,1.662770,-1.196354,1.840027,-0.417169,-0.281448,1.421067,-0.550460,-0.097977,-0.455820,...,-0.627539,-0.759820,-0.541482,-1.560930,-1.266146,0.591704,-0.739262,-0.988835,-0.797694,-0.976992


In [194]:
x_train = x_avg.join(x_std)
x_train_test = x_avg_test.join(x_std_test)

In [195]:
x_train.shape,y_train.shape,x_train_test.shape,y_train_test.shape

((182, 2048), (182, 1), (200, 512), (200, 1))

In [196]:
corr_matrix = x_train.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
x_train.drop(to_drop, axis=1, inplace=True)
# x_train_test.drop(to_drop, axis=1, inplace=True)

x_train.shape,x_train_test.shape

((182, 2016), (200, 512))

In [197]:
X_trains_df=pd.DataFrame(x_train,columns=x_train.columns)

In [198]:
y_train.head()

,MGMT Status
0,0
1,0
2,0
3,0
4,0


In [199]:
# print(X_trains_df.isnull().sum())
# print(X_trains_df.isin([np.nan, np.inf, -np.inf]).sum().sum())

# print(pd.isnull(y_train).sum())
# print(y_train.isin([np.nan, np.inf, -np.inf]).sum().sum())

In [200]:
# mean_val = X_trains_df['VGGNet_Std_F511'].mean()
# X_trains_df['VGGNet_Std_F511'] = X_trains_df['VGGNet_Std_F511'].fillna(mean_val)

## **SVM Feature Selection**

In [223]:
svm = SVC(kernel="linear")


In [234]:
# dtc_selector_model = RFE(estimator=dtc)
# dtc_selector_model_fit=dtc_selector_model.fit(X_trains_df,y_train.values.ravel())

svm_selector_model = RFE(estimator=svm,n_features_to_select=45)
svm_selector_model_fit=svm_selector_model.fit(X_trains_df,y_train.values.ravel())



In [235]:
feat_index = pd.Series(data = svm_selector_model_fit.ranking_, index = x_train.columns)
signi_feat_rfe = feat_index[feat_index==1].index
print('Significant features from RFE using SVM',signi_feat_rfe)

Significant features from RFE using SVM Index(['FLAIR_AlexNet_Avg_F44', 'FLAIR_AlexNet_Avg_F64',
       'FLAIR_AlexNet_Avg_F150', 'T1GD_AlexNet_Avg_F28',
       'T1GD_AlexNet_Avg_F175', 'T1GD_AlexNet_Avg_F184', 'T2_AlexNet_Avg_F8',
       'T2_AlexNet_Avg_F28', 'T2_AlexNet_Avg_F69', 'T2_AlexNet_Avg_F86',
       'T2_AlexNet_Avg_F146', 'T2_AlexNet_Avg_F180', 'T2_AlexNet_Avg_F224',
       'FLAIR_AlexNet_Std_F33', 'FLAIR_AlexNet_Std_F66',
       'FLAIR_AlexNet_Std_F150', 'FLAIR_AlexNet_Std_F161',
       'FLAIR_AlexNet_Std_F163', 'FLAIR_AlexNet_Std_F164',
       'FLAIR_AlexNet_Std_F210', 'FLAIR_AlexNet_Std_F212',
       'FLAIR_AlexNet_Std_F231', 'T1GD_AlexNet_Std_F29',
       'T1GD_AlexNet_Std_F60', 'T1GD_AlexNet_Std_F70', 'T1GD_AlexNet_Std_F113',
       'T1GD_AlexNet_Std_F157', 'T1GD_AlexNet_Std_F175',
       'T1GD_AlexNet_Std_F231', 'T1_AlexNet_Std_F26', 'T1_AlexNet_Std_F72',
       'T1_AlexNet_Std_F80', 'T1_AlexNet_Std_F132', 'T1_AlexNet_Std_F234',
       'T2_AlexNet_Std_F2', 'T2_AlexNet_

In [236]:
print('Original number of features present in the dataset : {}'.format(x_train.shape[1]))
print('Number of features selected by SVM using Recursive feature selection is : {}'.format(len(signi_feat_rfe)))

Original number of features present in the dataset : 2016
Number of features selected by SVM using Recursive feature selection is : 45


In [237]:
df_selected_features = x_train.filter(items=signi_feat_rfe, axis=1)
df_selected_features_test = x_train_test.filter(items=signi_feat_rfe, axis=1)

In [238]:
df_selected_features_test.shape

(200, 0)

In [239]:
df_selected_features.head()

,FLAIR_AlexNet_Avg_F44,FLAIR_AlexNet_Avg_F64,FLAIR_AlexNet_Avg_F150,T1GD_AlexNet_Avg_F28,T1GD_AlexNet_Avg_F175,T1GD_AlexNet_Avg_F184,T2_AlexNet_Avg_F8,T2_AlexNet_Avg_F28,T2_AlexNet_Avg_F69,T2_AlexNet_Avg_F86,...,T2_AlexNet_Std_F44,T2_AlexNet_Std_F46,T2_AlexNet_Std_F53,T2_AlexNet_Std_F59,T2_AlexNet_Std_F89,T2_AlexNet_Std_F132,T2_AlexNet_Std_F143,T2_AlexNet_Std_F186,T2_AlexNet_Std_F233,T2_AlexNet_Std_F235
0,0.145970,0.186520,0.014388,1.625550,-0.139742,-0.399418,1.426487,0.320444,-0.527613,-1.095693,...,1.262691,-0.105284,-0.554188,1.544819,0.267263,-0.205949,0.060011,0.473839,-0.260478,-0.235157
1,-1.220691,1.402888,-0.401761,-1.214155,-0.380374,-0.515021,-1.052013,-0.888498,1.365019,0.941180,...,-1.145513,1.393090,-0.596792,-0.637709,2.265109,-0.088093,0.035673,-0.679071,-0.229409,-1.067429
2,-0.248483,1.842285,-0.310561,0.685064,-0.043951,0.717741,0.102474,0.054189,-0.656491,-0.522976,...,-0.572308,0.423557,-0.517404,0.086820,1.674814,-0.536037,0.521160,1.220994,1.476906,-0.264977
3,-1.427289,0.210814,-0.868475,-0.765876,-0.507019,0.995635,0.678640,-0.305453,-0.330748,-0.584999,...,-0.461241,-0.676667,0.754885,-0.962456,0.886408,0.452672,-0.058776,0.859011,0.755652,0.675905
4,-0.521739,2.307603,-0.176986,-0.538550,0.571977,-0.361254,-0.958385,-0.651897,-0.232137,-1.381912,...,-0.685816,0.083778,0.246441,0.419412,-0.767924,0.137810,0.381280,-0.269043,0.592335,1.085459


In [240]:
df_selected_features.to_csv('SelectedFeaturesAlexFinalMGMT.csv',index = None,header = df_selected_features.columns)
df_selected_features_test.to_csv('SelectedFeaturesAlexFinalTESTOS.csv',index = None,header = df_selected_features_test.columns)



In [241]:
# skip NN 21 March
columns = ['Alex_F' + str(i) for i in range(df_selected_features.shape[1])]
# df_selected_features.to_csv('SelectedFeaturesAlexFinalMergedMGMT30.csv',index = None,header = df_selected_features.columns)
df_selected_features.to_csv('Alex45MGMT.csv',index = None,header = df_selected_features.columns)
columns = ['Alex_F' + str(i) for i in range(df_selected_features_test.shape[1])]
df_selected_features_test.to_csv('SelectedFeaturesAlexFinalMergedTESTOS.csv',index = None,header = columns)

In [232]:
df_selected_features.shape

(182, 90)

In [233]:
df_selected_features

,FLAIR_AlexNet_Avg_F6,FLAIR_AlexNet_Avg_F44,FLAIR_AlexNet_Avg_F64,FLAIR_AlexNet_Avg_F66,FLAIR_AlexNet_Avg_F76,FLAIR_AlexNet_Avg_F118,FLAIR_AlexNet_Avg_F129,FLAIR_AlexNet_Avg_F142,FLAIR_AlexNet_Avg_F150,FLAIR_AlexNet_Avg_F184,...,T2_AlexNet_Std_F89,T2_AlexNet_Std_F112,T2_AlexNet_Std_F132,T2_AlexNet_Std_F143,T2_AlexNet_Std_F186,T2_AlexNet_Std_F188,T2_AlexNet_Std_F224,T2_AlexNet_Std_F233,T2_AlexNet_Std_F235,T2_AlexNet_Std_F246
0,-0.459453,0.145970,0.186520,-0.655547,-0.394447,1.239120,-0.488738,-0.904459,0.014388,0.541280,...,0.267263,0.685745,-0.205949,0.060011,0.473839,1.560603,0.609170,-0.260478,-0.235157,0.057928
1,-0.741583,-1.220691,1.402888,-1.193737,-0.303580,-0.712476,-0.944688,2.029610,-0.401761,-0.894586,...,2.265109,-0.169823,-0.088093,0.035673,-0.679071,-0.389342,-1.089219,-0.229409,-1.067429,-0.689153
2,-0.407710,-0.248483,1.842285,-0.172628,0.274792,-0.632192,-0.673704,0.554625,-0.310561,0.349423,...,1.674814,0.095485,-0.536037,0.521160,1.220994,0.133278,-0.107408,1.476906,-0.264977,-0.850276
3,-1.695235,-1.427289,0.210814,-0.969057,-0.394447,-0.186965,-0.383104,-0.253265,-0.868475,0.446160,...,0.886408,-0.685613,0.452672,-0.058776,0.859011,-1.307627,1.385982,0.755652,0.675905,1.634048
4,0.698145,-0.521739,2.307603,-0.895098,0.276605,-0.582117,-0.509365,1.675809,-0.176986,-0.525231,...,-0.767924,-1.078808,0.137810,0.381280,-0.269043,-1.232517,-0.540684,0.592335,1.085459,-0.627539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,-0.131227,0.123707,-0.172675,1.550663,-0.394447,-0.297923,1.062508,-0.175098,2.410070,-0.569756,...,0.532629,0.624281,-0.071382,-0.293421,-0.679071,0.535226,0.438403,-0.467925,0.377994,0.803811
178,0.741646,0.469325,1.392680,1.933344,-0.345050,-0.863643,0.989592,-0.503343,-0.054716,-0.842742,...,1.065362,0.993159,0.579387,0.182564,-0.679071,-1.453585,-0.460994,2.118021,0.733143,-0.880549
179,0.582092,1.534637,0.686524,0.156636,-0.394447,0.774858,0.581722,0.422849,-0.628840,1.324781,...,-0.523987,-0.435889,-1.351015,2.545929,-0.268603,0.675825,1.771289,-0.272034,0.499175,-0.155916
180,0.296515,1.384878,-0.671920,-0.692693,-0.391783,-0.723944,0.838706,-0.052133,-0.758672,-0.738315,...,-0.808980,-1.502912,-1.046933,-1.180261,-0.679071,-1.683312,-1.229692,-0.736345,-1.383397,-0.717767


In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC
import numpy as np

# define the SVM classifier
svm = SVC()

# define the parameter grid to search over
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.1, 0.01, 0.001],
    'kernel': ['linear', 'rbf']
}

# create the grid search object
grid_search = GridSearchCV(svm, param_grid, cv=StratifiedKFold(n_splits=5), scoring='roc_auc')

# fit the grid search to the data
grid_search.fit(df_selected_features, y.values.ravel())

# print the best parameters and corresponding score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import numpy as np

# define the SVM classifier with the best parameters found by GridSearchCV
svm = SVC(C=grid_search.best_params_['C'], gamma=grid_search.best_params_['gamma'], kernel=grid_search.best_params_['kernel'])

# define the stratified k-fold cross-validator
cv = StratifiedKFold(n_splits=5)

# define empty lists to store the true positive rate (tpr) and false positive rate (fpr) for each fold
tpr_list = []
fpr_list = []

# loop over each fold
for train_idx, test_idx in cv.split(df_selected_features, y):
    X_train, X_test = df_selected_features.iloc[train_idx], df_selected_features.iloc[test_idx]
    y_train_fold, y_test_fold = y.iloc[train_idx], y.iloc[test_idx]
    
    # fit the SVM classifier to the training data and predict on the test data
    svm.fit(X_train, y_train_fold.values.ravel())
    y_pred = svm.predict(X_test)
    
    # calculate the false positive rate (fpr) and true positive rate (tpr) using the roc_curve function
    fpr, tpr, thresholds = roc_curve(y_test_fold, y_pred)
    tpr_list.append(tpr)
    fpr_list.append(fpr)
    
    roc_auc = auc(fpr, tpr)
    print('Fold %d ROC AUC: %.4f' % (i+1, roc_auc))

# compute the mean and standard deviation of the tpr and fpr across all folds
mean_tpr = np.mean(tpr_list, axis=0)
mean_fpr = np.mean(fpr_list, axis=0)
std_tpr = np.std(tpr_list, axis=0)
std_fpr = np.std(fpr_list, axis=0)

# compute the area under the ROC curve (AUC-ROC)
auc_roc = auc(mean_fpr, mean_tpr)

# plot the ROC curve with shaded standard deviation
plt.plot(mean_fpr, mean_tpr, color='b', label='Mean ROC (AUC = %0.2f)' % auc_roc)
plt.fill_between(mean_fpr, mean_tpr - std_tpr, mean_tpr + std_tpr, alpha=0.2, color='b')
plt.fill_betweenx(mean_tpr, mean_fpr - std_fpr, mean_fpr + std_fpr, alpha=0.2, color='b')
plt.plot([0, 1], [0, 1], linestyle='--', color='grey', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()


In [ ]:
# Step 6: Train an SVM model using Stratified Kfold cross validation and grid search cv
scaler = StandardScaler()
svc = SVC(probability=True)
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto']
}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

mean_fpr = np.linspace(0, 1, 100)
tprs = []
aucs = []
best_svc = None
best_score = 0

for i, (train, test) in enumerate(cv.split(df_selected_features, y)):
    X_train = df_selected_features.iloc[train]
    y_train = y.iloc[train].values.ravel()
    X_test = df_selected_features.iloc[test]
    y_test = y.iloc[test].values.ravel()
    
#     common_indices = set(train).intersection(set(test))
#     print('Common indices:', common_indices if common_indices else None)

    grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=cv, scoring='roc_auc')
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)
    grid_search.fit(X_train_scaled, y_train)
    best_svc_fold = grid_search.best_estimator_
    
    score = grid_search.best_score_
    if score > best_score:
        best_score = score
        best_svc = grid_search.best_estimator_
    
    y_score = best_svc_fold.predict_proba(X_test_scaled)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_score)
    tprs.append(np.interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    print('Fold %d ROC AUC: %.4f' % (i+1, roc_auc))


plt.figure(figsize=(8,6))
for i in range(len(tprs)):
    plt.plot(mean_fpr, tprs[i], lw=2, alpha=0.6, label='ROC fold %d (AUC = %0.2f)' % (i+1, aucs[i]))

plt.plot([0,1],[0,1],'--',lw=2, color='black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b', label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),lw=2, alpha=.8)

# plt.fill_between(mean_fpr, mean_tpr - std_auc, mean_tpr + std_auc, alpha=.2, color='b')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.savefig('auc_roc.png')
plt.show()


In [ ]:
from sklearn.metrics import classification_report, plot_roc_curve

# Predict on test data
y_pred = best_svc.predict(df_selected_features_test)

# Print classification report
print(classification_report(y_train_test, y_pred))